In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time
import os
from itertools import product

# RPY2 is used an interconnect between Python and R. It allows
# my to run R code from python which makes this experimentation
# process smoother.
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats') # standard regression package
matching = importr('Matching') # GenMatch package
snow = importr('snow') # cluster manager

## 1. Define an Experimental Framework

### A. Cluster Compute

See the simulations file for details on this cluster compute code

In [2]:
AWS_MASTER_DNS="ip-172-31-42-147.ec2.internal"
AWS_SLAVE_1 = "ubuntu@ip-172-31-43-193.ec2.internal"
AWS_SLAVE_2 = "ubuntu@ip-172-31-81-244.ec2.internal"
AWS_MASTER_PORT_RANGE = list(range(11305, 11340))

class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, ports=None):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            # Set the acceptable ports for connection
            # from the slaves
            if not ports:
                ports = AWS_MASTER_PORT_RANGE
            
            # Construct the connection string
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
                
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(ports), master=AWS_MASTER_DNS, outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [3]:
# Local cluster
cluster_provider = ClusterProvider(n_nodes=15)

In [4]:
# Remote cluster
# cluster_provider = ClusterProvider(remote_hosts=[(AWS_SLAVE_1, 8)],
#                                     ports = list(range(11305, 11314)))

In [5]:
# Run this with True to kill the cluster
kill = False # termination protection
if kill:
    cluster_provider.kill_cluster()

### B. Flexible Causal Inference Methods

#### Estimators

In this file, we copy across the GenMatch and Logistic Propensity score methods implemented in the simulations file. See the Simulations notebook for details on these functions

In [6]:
# Helper function which runs logistic regression in R
# to determine propensity scores for a dataset. This is used
# in the propensity score matching method and in GenMatch
def get_propensity_scores(assignments, covariate_data):
    # Setup
    print(assignments.shape, covariate_data.shape)
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [7]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data, *args, **kwargs):
    global gm_warnings
    logistic_propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Use prop scores from the neural network regression
    # if supplied
    nn_p_scores = kwargs.get("nn_p_scores", None)
    if nn_p_scores is not None:
        if gm_warnings:
            print("Using p-scores from neural net")
        propensity_scores = nn_p_scores
    else:
        propensity_scores = logistic_propensity_scores
    
    # Run matching on prop scores
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    gm_warnings = False # only warn once
    return (np.array(match_out.rx2("est").rx(1,1))[0], None)

In [26]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data, *args, **kwargs):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Flag on whethert or not to use propensity scores
    # in GenMatch
    if kwargs.get("genmatch_with_prop_scores", True):
        print("Genmatch running with p scores")
        
        # Parameter to allow prop scores to
        # be derived from custom data
        propensity_vars = kwargs.get("propensity_vars", None)
        if propensity_vars is None:
            print("Finding propensity score on matching vars")
            propensity_vars = covariate_data
        else:
            print("Finding propensity scores with custom vars")

        logistic_p_scores = np.array(get_propensity_scores(assignments, propensity_vars))
        
       
        nn_p_scores = kwargs.get("nn_p_scores", None)
        print("KWARGS P", nn_p_scores)
        # Use either the prop scores from the neural or the logistic regression
        # or both!
        if (nn_p_scores is not None) and kwargs.get("nn_p_scores_with_logistic", False):
            print("Using neural net and logistic p scores")
            matching_data = np.hstack([covariate_data, nn_p_scores.reshape(-1, 1),
                                       logistic_p_scores.reshape(-1, 1)])
            
        elif (nn_p_scores is not None):
            print("Using neural net  p scores")
            print(nn_p_scores.shape, covariate_data.shape)
            matching_data = np.hstack([covariate_data, nn_p_scores.reshape(-1, 1)])
        else:
            print("Using logistic p scores")
            matching_data = np.hstack([covariate_data, logistic_p_scores.reshape(-1, 1)])
             
    else:
        print("Not using prop scores")
        matching_data = covariate_data
    
    # Allow evaluation of balance on custom vars
    balance_vars = kwargs.get("balance_vars", None)
    if balance_vars is None:
        print("Evaluating balance on matching covars")
        balance_vars = covariate_data
    else:
        print("Evaluating balance on custom vars")
    
    # Run GenMatch
    print("Matching data shape:", matching_data.shape)
    print("Balance vars shape:", balance_vars.shape)
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=balance_vars,
        wait_generations=5,
        pop_size=1000,
        print_level=0,
        cluster=cl)
    
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=matching_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0], \
            np.array(gen_out.rx2("value")), \
            np.diag(np.array(gen_out.rx2("Weight.matrix")))

In [9]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [10]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

### C. Data Interconnect

CSV files are used to pass information between this file and the Neural Network files. The code below defines helper functions which save data generated in this file to CSVs for neural net training and  functions which read in the processed data which results from the training. 

In [11]:
RAW_DATA_DIR = "../Data/NSW/Raw/"
PROCESSED_DATA_DIR = "../Data/NSW/Processed/"
VAE = "VAE/"
REG = "Regression/"

In [12]:
# Create assignment, outcome and covar data files based on
# the data available at http://users.nber.org/~rdehejia/data/nswdata2.html
def process_raw_nsw_files():
    treated = np.loadtxt(RAW_DATA_DIR + "/nswre74_treated.txt", delimiter="  ", dtype=str)[:, 1:].astype(float)
    control = np.loadtxt(RAW_DATA_DIR + "/nswre74_control.txt", delimiter="  ", dtype=str)[:, 1:].astype(float)
    cps_control = np.loadtxt(RAW_DATA_DIR + "/cps_controls.txt", delimiter="  ", dtype=str)[:, 1:].astype(float)

    experimental_data = np.vstack([treated, control])
    experimental_assignments = experimental_data[:, 0]
    experimental_outcomes = experimental_data[:, 9]
    experimental_covars = experimental_data[:, 1:9]

    exp_effect = np.mean(experimental_outcomes[experimental_assignments==1]) - \
        np.mean(experimental_outcomes[experimental_assignments==0])
    print("Exp treat effect:", exp_effect)
    assert(int(exp_effect) == 1794)
    
    np.savetxt(RAW_DATA_DIR + "nsw74_experiment_assignments.csv", experimental_assignments, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_experiment_outcomes.csv", experimental_outcomes, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_experiment_covars.csv", experimental_covars, delimiter=",")
    
    
    observational_data = np.vstack([treated, cps_control])
    observational_assignments = observational_data[:, 0]
    observational_outcomes = observational_data[:, 9]
    observational_covars = observational_data[:, 1:9]
    
    obs_effect = np.mean(observational_outcomes[observational_assignments==1]) - \
        np.mean(observational_outcomes[observational_assignments==0])
    print("Obs treat effect:", obs_effect)
    
    np.savetxt(RAW_DATA_DIR + "nsw74_all_assignments.csv", observational_assignments, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_all_outcomes.csv", observational_outcomes, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_all_covars.csv", observational_covars, delimiter=",")
    
    return exp_effect

# Retrieve processed and unprocessed data from files in order to run experiments. 
def get_data_from_file(loss_type=None, nn_p_regression=False, original_mode=False):
    
    original_covariates = np.loadtxt(RAW_DATA_DIR + "nsw74_all_covars.csv", delimiter=",")
    outcomes = np.loadtxt(RAW_DATA_DIR + "nsw74_all_outcomes.csv", delimiter=",")
    assignments = np.loadtxt(RAW_DATA_DIR + "nsw74_all_assignments.csv", delimiter=",")
    
    extra_data = {}
    if not (loss_type or nn_p_regression or original_mode):
        raise Exception("Invalid config. Need loss type, p regression, or original mode option")
    
    if not original_mode:
        if loss_type:
            if loss_type in ["reconstruction", "sparsity"]:
                covariate_file_name = PROCESSED_DATA_DIR + "nsw74_all_covars_{}.csv".format(loss_type)
                covariates = np.loadtxt(covariate_file_name, delimiter=",")

            elif loss_type in ["vae"]:
                covariate_file_name = PROCESSED_DATA_DIR + VAE + "nsw74_all_covars.csv"
                covariates_with_std = np.loadtxt(covariate_file_name, delimiter=",")

                # Split means and covariance
                column_count = covariates_with_std.shape[1]
                covar_marker = int(column_count/2)

                covariates = covariates_with_std[:, :covar_marker]
                extra_data["covariate_covariance"] = covariates_with_std[:, covar_marker:]

            if not nn_p_regression:
                # If we aren't accessing NN prop scores, return now.
                return assignments, outcomes, covariates, original_covariates, extra_data

        if nn_p_regression:
            reg_file_name = PROCESSED_DATA_DIR + REG + "nsw74_all_covars.csv"
            regression_prop_scores = np.loadtxt(reg_file_name, delimiter=",")

            extra_data["nn_p_scores"] = regression_prop_scores

            if loss_type:
                return assignments, outcomes, covariates, original_covariates, extra_data
    else:
        # Add interaction and quadratic terms
        var_numbers = list(range(8))
        for i in var_numbers:
            new_var = original_covariates[:, i]**2
            original_covariates = np.hstack([original_covariates, new_var.reshape(-1, 1)])
            
        for i, j in combinations(var_numbers, 2):
            new_var = original_covariates[:, i]*original_covariates[:, j]
            original_covariates = np.hstack([original_covariates, new_var.reshape(-1, 1)])
    
    # Original data path
    return assignments, outcomes, original_covariates, extra_data

In [13]:
# Write data files for 1000 runs all models
# Careful with this, it writes ~3GB of data. 
process_files = True
if process_files:
    nsw_effect = process_raw_nsw_files()

Exp treat effect: 1794.342404270271
Obs treat effect: -8497.516142636992


In [14]:
assignments, outcomes, covariates, original_covariates, extra_data = \
get_data_from_file(loss_type="reconstruction")
print(covariates.shape)
covariates

(16177, 4)


array([[-9.43083882e-01,  5.36252937e+01,  4.08237000e+01,
        -7.54744949e+01],
       [-2.20184159e+00,  3.22563629e+01,  2.60338554e+01,
        -4.57855377e+01],
       [-3.23585248e+00,  4.28735809e+01,  3.54630890e+01,
        -5.97982407e+01],
       ...,
       [-8.55814453e+02,  1.31106848e+03,  1.09766016e+03,
         1.58020435e+03],
       [-5.30705029e+03,  1.02402266e+04,  1.05055029e+04,
         1.43952393e+04],
       [-7.35459766e+03,  1.04665645e+04,  8.29260254e+03,
         1.30563252e+04]])

Utilities to store and retrive pickled results dictionaries. This allows us to persist and sync results across different machines/processes.

In [15]:
def store_results_dict(results, name):
    pickle.dump(results, open("../Results/NSW/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "../Results/NSW/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### D. Experiment Runner Code

This code is largely a duplication of the the code in the Simulations file, but simplified for a single run on the NSW dataset.

#### Single Simulation

In [16]:
# Wrapper function for all three of the matching methods above. This allows the matching function
# to be passed into the experiment running code without concern over the method API.
def get_estimate(outcomes, assignments, covar_data, method, *args, **kwargs):
    return method(outcomes, assignments, covar_data, *args, **kwargs)

In [17]:
def run_simulation(estimator=logistic_prop_matching_est,
                   store_name=None,
                   overwrite=False,
                   verbose=True,
                   *args, **kwargs):
    
    global gm_warnings
    gm_warnings = True
    
    result_data = None
    
    if store_name:
        result_data = retrieve_results_dict(store_name)
    
    if overwrite or (not result_data):
        print("No valid, existant results found. Beggining trial.\n")
    
        # Prepare config for matching estimators
        balance_vars = None
        propensity_vars = None
        extra_data = {}

        # Prepare data for matching
        loss_type = kwargs.get("loss_type", None)
        nn_p_regression = kwargs.get("nn_p_regression", None)
        original_mode = kwargs.get("original_mode", None)

        if not (loss_type or nn_p_regression or original_mode):
            raise Exception("Must supply loss type or p regression option to read from files")

        if loss_type:
            assignments, outcomes, covar_data, original_covars, extra_data = \
                get_data_from_file(loss_type=loss_type, nn_p_regression=nn_p_regression)

            if kwargs.get("evaluate_on_original_covars", False):
                balance_vars=original_covars

            if kwargs.get("propensity_on_original_covars", False):
                propensity_vars=original_covars

        else:
            assignments, outcomes, covar_data, extra_data =\
                get_data_from_file(
                    loss_type=loss_type,
                    nn_p_regression=nn_p_regression,
                    original_mode=original_mode)

        # Run matching
        print(extra_data)
        result_tuple  = get_estimate(outcomes,
                                  assignments,
                                  covar_data,
                                  estimator,
                                  balance_vars=balance_vars,
                                  propensity_vars=propensity_vars,
                                  *args,
                                  **extra_data,
                                  **kwargs)
            
        result = result_tuple[0]
        
        bias = np.abs((nsw_effect-result)/nsw_effect * 100)

        result_data = {"result": result, "Bias": bias}
        
        if estimator == genmatch_est:
            result_data["fitnesses"] = result_tuple[1]
            result_data["weights"] = result_tuple[2]
            
            if verbose:
                print("Fitnesses: ", result_data["fitnesses"][:10])
                print("Weights: ",  result_data["weights"])
        
        if store_name:
            store_results_dict(result_data, store_name)
    else:   
        if verbose:
            print("Displaying cached results.\n")
    
    if verbose:
        print("Bias", result_data["Bias"])
        print("Estimated effect ${}".format(result_data["result"]))
        print("Min p:",result_data["fitnesses"][0])
        
    return result_data

### Vanilla GenMatch

In [18]:
_ = run_simulation(
    estimator=genmatch_est,
    store_name="original_mode",
    original_mode=True,
    verbose=True)

Displaying cached results.

Bias 7.53552590659057
Estimated effect $1929.5555409969973
Min p: 0.18837925553825718


In [19]:
_ = run_simulation(
    estimator=genmatch_est,
    store_name="original_mode_2",
    original_mode=True,
    verbose=True)

Displaying cached results.

Bias 5.202436597037277
Estimated effect $1887.691930186186
Min p: 0.20652956976381498


In [20]:
_ = run_simulation(
    estimator=genmatch_est,
    store_name="original_mode_3",
    original_mode=True,
    verbose=True)

Displaying cached results.

Bias 4.697325755514021
Estimated effect $1878.6285121681678
Min p: 0.2102924241804689


### AE with only Reconstruction Loss


#### Config 1

Pure reconstruction without propensity score estimates

In [21]:
_ =run_simulation(
    estimator=genmatch_est,
    loss_type="reconstruction",
    store_name="AE_reconstruction_plain",
    genmatch_with_prop_scores=False,
    verbose=True)

Displaying cached results.

Bias 10.552266558362124
Estimated effect $1604.9986108019482
Min p: 7.901347900407529e-07


#### Config 2
Pure reconstruction *with* propensity score estimateson uncompressed

In [22]:
_ = run_simulation(
    estimator=genmatch_est,
    loss_type="reconstruction",
    store_name="AE_reconstruction_withp",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    verbose=True)

Displaying cached results.

Bias 66.44590893591551
Estimated effect $602.0752843303297
Min p: 0.36341567821079


#### Config 3
Pure reconstruction, with propensity scores on uncompressed, evaluating balance on uncompressed data

In [23]:
_ = run_simulation(
    estimator=genmatch_est,
    loss_type="reconstruction",
    store_name="AE_reconstruction_withp_evalonoriginal",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    evaluate_on_original_covars=True,
    verbose=True)

Displaying cached results.

Bias 6.63547942007092
Estimated effect $1913.4056252312305
Min p: 0.022861744110306503


In [24]:
_ = run_simulation(
    estimator=genmatch_est,
    loss_type="reconstruction",
    store_name="AE_reconstruction_withp_evalonoriginalX",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    evaluate_on_original_covars=True,
    verbose=True)

Displaying cached results.

Bias 5.9810171457491235
Estimated effect $1901.6623311231228
Min p: 0.013653451239564962


### Neural Propensity Score Regression

In [27]:
run_simulation(
    estimator=genmatch_est,
    loss_type="reconstruction",
    store_name="AE_reconstruction_withnn_prop",
    nn_p_regression=True,
    verbose=True)

Displaying cached results.

Bias 27.736597754240222
Estimated effect $1296.652869264063
Min p: 0.3241284907834834


{'Bias': 27.736597754240222,
 'fitnesses': array([0.32412849, 0.37165754, 0.39241145, 0.39907434, 0.43998404,
        0.43998404, 0.76058847, 0.82152893]),
 'result': 1296.652869264063,
 'weights': array([2.72854650e+02, 8.45744194e+02, 8.49180523e+02, 9.42477681e+02,
        2.34662677e-02])}